# Team Harvey Wiley Nutritional Adequacy Notebook
In this notebook, we examine the nutritional adequacy of the diets of the housholds in our analysis.

**Usage Note - Updating Notebook**:
- If you make changes locally to your version of the notebook, you will need to manually update the notebook via the following steps
    1. Uncomment the code in the cell below and run it.
    2. Restart the notebook's kernel.
    3. Refresh the page. <br>
    **Warning** - Make sure that the aforementioned code is commented once you successfully updated the notebook.

In [26]:
### Uncomment this code to manually update the notebook
# !git reset --hard origin/main

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, widgets
import fooddatacentral as fdc

## Examing Nutritional Adequacy

In [227]:
diet_refs = pd.read_csv('./data/uganda/rdi.csv')
dr_categories = diet_refs['n'].values

In [ ]:
def get_data(country, year):
    '''
    Description
    --------------------------------------------------
    This function gets all of the datasets needed for
    the analysis.
    
    Inputs
    --------------------------------------------------
    + country : string; country of interest
    + year : string; year of interest - in the form
            20XX-XX+1
    
    Outputs
    --------------------------------------------------
    + food_nutrients : pandas dataframe; contains the
            nutritional contents of all foods of 
            interest
    + food_prices : pandas dataframe; contains the
            prices of all foods of interest across 
            different years
    + hh_chars : pandas dataframe; contains key
            characteristics of individual households
            across different years
    + expenditures : pandas dataframe; contains food
            expenditures of individual households
            across different years
    + hhc_sub : pandas dataframe; a filtered version
            of hh_chars that only contains entries 
            from the year of interest
    + fp_sub : pandas dataframe; a filtered version
            of food_prices that only contains 
            entries from the year of interest
    + food_cols : numpy array; all of the foods of 
            interest for the analysis
    + fp_sub_avgs : pandas dataframe; column-wise 
            averages of food prices in fp_sub
            across all regions
    '''
    food_nutrients = pd.read_csv(f'./data/{country}/fct.csv')#.set_index('n')
    food_prices = pd.read_csv(f'./data/{country}/food_prices.csv').fillna(0)
    hh_chars = pd.read_csv(f'./data/{country}/hh_chars.csv')

    ### Deals With Housholds from an  Unknown Region
    hh_chars['m'] = hh_chars['m'].fillna('Unknown')
    expenditures = pd.read_csv(f'./data/{country}/expenditures_{year_range[-5:]}.csv').fillna(0)
    
    hhc_sub = hh_chars[hh_chars['t'] == year_range].reset_index(drop = True)
    fp_sub = food_prices[food_prices['t'] == year_range].set_index('m').drop(columns = ['t'])
    food_cols = expenditures.iloc[0:2, 3:].columns
    fp_sub_avgs = fp_sub.reset_index(drop = True)
    fp_sub_avgs = pd.DataFrame(fp_sub_avgs.mean()).rename(columns = {0 : 'Mean_Price'})
    
    return food_nutrients, food_prices, hh_chars, expenditures, hhc_sub, fp_sub, food_cols, fp_sub_avgs

In [270]:
def get_col_counts(expenditures_df, food_col, prices, price_avs):
    '''
    Description
    --------------------------------------------------
    This is a function that gets the food expenditure 
    counts for each household for a specific food.
    
    Inputs
    --------------------------------------------------
    + expenditures_df : pandas dataframe; contains food
            expenditures of individual households
            across different years
    + food_col : string; food to get count for
    + prices : pandas dataframe; contains the
            prices of all foods of interest across 
            different years
    + price_avgs : pandas dataframe; column-wise 
            averages of food prices in fp_sub
            across all regions
    
    Outputs
    --------------------------------------------------
    + counts : numpy array; food expenditure counts for 
            each household
    '''
    counts = []
    for idx in expenditures_df.index:
        region = expenditures_df.loc[idx, 'm']
        expenditure = expenditures_df.loc[idx, food_col]
        if region == 'Unknown':
            # Imputes price from unknown region with the mean accross all regions
            price = fp_sub_avgs.loc[food_col][0]
            if price == 0: # Don't want to divide by zero
                count = 0
                counts.append(count)
            else:
                count = expenditure / price
                counts.append(count)   
        else:
            price = prices.loc[region, food_col]
            if price == 0: # Don't want to divide by zero
                count = 0
                counts.append(count)
            else:
                count = expenditure / price
                counts.append(count)
    return counts

In [28]:
def get_counts(expenditures_df, food_cols, prices, price_avs):
    '''
    Description
    --------------------------------------------------
    This is a function that gets the food expenditure 
    counts for each household for all the foods and
    puts it into a pandas dataframe.
    
    Inputs
    --------------------------------------------------
    + expenditures_df : pandas dataframe; contains food
            expenditures of individual households
            across different years
    + food_cols : numpy array; all of the foods of 
            interest for the analysis
    + prices : pandas dataframe; contains the
            prices of all foods of interest across 
            different years
    + price_avgs : pandas dataframe; column-wise 
            averages of food prices in fp_sub
            across all regions
    
    Outputs
    --------------------------------------------------
    + counts : pandas dataframe; food expenditure counts for 
            each household
    '''
    count_df = expenditures_df.drop(columns = food_cols)
    for food_col in food_cols:
        counts = get_col_counts(expenditures_df, food_col, prices, price_avs)
        count_df[food_col] = counts
    return count_df

In [256]:
def get_diet_adequacies(food_nutrients, food_prices, hh_chars, 
                    expenditures, hhc_sub, fp_sub, food_cols, fp_sub_avgs, diet_refs):
    '''
    Description
    --------------------------------------------------
    This is a function that gets the food expenditure 
    adequacies for each household for all the foods and
    puts it into a pandas dataframe.
    
    Inputs
    --------------------------------------------------
    + food_nutrients : pandas dataframe; contains the
            nutritional contents of all foods of 
            interest
    + food_prices : pandas dataframe; contains the
            prices of all foods of interest across 
            different years
    + hh_chars : pandas dataframe; contains key
            characteristics of individual households
        across different years
    + expenditures : pandas dataframe; contains food
            expenditures of individual households
            across different years
    + hhc_sub : pandas dataframe; a filtered version
            of hh_chars that only contains entries 
            from the year of interest
    + fp_sub : pandas dataframe; a filtered version
            of food_prices that only contains 
            entries from the year of interest
    + food_cols : numpy array; all of the foods of 
            interest for the analysis
    + fp_sub_avgs : pandas dataframe; column-wise 
            averages of food prices in fp_sub
            across all regions
    + diet_refs : pandas dataframe; dietary reference 
            intakes for different age-sex groups
    
    Outputs
    --------------------------------------------------
    + hh_diet_adequacy : pandas dataframe; contains
            the nutrient shares where share equals
            (household nutrient consumption) / (hous-
            -ehold nutrition requirement)
    + household_master : pandas dataframe; contains
            the expenditure counts & 
            characteristics for each household
    + nutrient_cols : numpy array; all of the
            column names of the nutrients of 
            interest
    '''
    exp_counts = get_counts(expenditures, food_cols, fp_sub, fp_sub_avgs)
    exp_counts['i'] = exp_counts['i'].astype(str)
    
    household_master = hhc_sub.merge(exp_counts, left_on = ['i', 't', 'm'], right_on = ['i', 't', 'm'])
    hhc_sub_num = hhc_sub.set_index('i').iloc[:, 2:-1]
    
    # If expenditures weekly, extrapolation_constant = 7
    # If expenditures daily, extrapolation_constant = 1
    extrapolation_constant = 7

    diet_refs_rev = diet_refs.set_index('n')
    diet_refs_rev = diet_refs_rev[hhc_sub_num.columns] # Reorders columns
    diet_refs_rev = extrapolation_constant * diet_refs_rev
    
    hh_diet_reqs = hhc_sub_num @ diet_refs_rev.T
    
    hh_consumption_cols = list(food_cols.to_numpy())
    hh_consumption_cols.append('i')

    hh_consumption = household_master[hh_consumption_cols].set_index('i')
    
    fn_rev = food_nutrients[food_nutrients['j'].isin(hh_consumption.columns)].rename(columns = {'j' : 'Food'}).set_index('Food')

    hh_consumption_rev = hh_consumption[fn_rev.index.to_numpy()]

    hh_consumption_nutrients = hh_consumption_rev @ fn_rev
    
    common_columns = hh_consumption_nutrients.columns.intersection(hh_consumption_nutrients.columns)
    common_rows = set(hh_consumption_nutrients.index).intersection(hh_consumption_nutrients.index)

    ### Gets common columns
    hh_consumption_nutrients_rev = hh_consumption_nutrients[common_columns]
    hh_diet_reqs_rev = hh_diet_reqs[common_columns]

    ### Gets common rows
    hh_consumption_nutrients_rev = hh_consumption_nutrients_rev[hh_consumption_nutrients_rev.index.isin(common_rows)]
    hh_diet_reqs_rev = hh_diet_reqs_rev[hh_diet_reqs_rev.index.isin(common_rows)]
    
    hh_diet_adequacy = hh_consumption_nutrients_rev / hh_diet_reqs_rev
    nutrient_cols = hh_diet_adequacy.columns
    
    hh_diet_adequacy['adequate_diet'] = (hh_diet_adequacy >= 1).sum(axis=1) / len(nutrient_cols)
    hh_diet_adequacy['adequate_diet_75pct'] = (hh_diet_adequacy >= 0.75).sum(axis=1) / len(nutrient_cols)
    hh_diet_adequacy['adequate_diet_50pct'] = (hh_diet_adequacy >= 0.5).sum(axis=1) / len(nutrient_cols)
    hh_diet_adequacy = hh_diet_adequacy[~hh_diet_adequacy.isin([np.inf]).any(axis=1)] # Drops infinite values
    
    return hh_diet_adequacy, household_master, nutrient_cols

## Plotting Nutritional Accuracy

In [ ]:
def get_plots(hh_diet_adequacy, household_master, nutrient_cols):
    '''
    Description
    --------------------------------------------------
    This is a function that compiles all of the plots
    for the analysis
    
    Inputs
    --------------------------------------------------
    + hh_diet_adequacy : pandas dataframe; contains
            the nutrient shares where share equals
            (household nutrient consumption) / (hous-
            -ehold nutrition requirement)
    + household_master : pandas dataframe; contains
            the expenditure counts & 
            characteristics for each household
    + nutrient_cols : numpy array; all of the
            column names of the nutrients of 
            interest
    
    Outputs
    --------------------------------------------------
    + Plots are displayed
    '''
    ### Distribution of Household Diet Adequacy Shares
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    sns.histplot(data = hh_diet_adequacy, x = 'adequate_diet', 
                 stat='density', kde = True, color = 'green', ax = axs[0])
    sns.histplot(data = hh_diet_adequacy, x = 'adequate_diet_75pct', 
                 stat='density', kde = True, color = 'orange', ax = axs[1])
    sns.histplot(data = hh_diet_adequacy, x = 'adequate_diet_50pct', 
                 stat='density', kde = True, color = 'red', ax = axs[2])

    plt.suptitle('Distribution of Household Diet Adequacy Shares at Different Adequacy Levels \n Share = (Nutrients At or Above Adequacy Level / Total Nutrients)')

    axs[0].set_title('100% Adequacy Level')
    axs[1].set_title('75% Adequacy Level')
    axs[2].set_title('50% Adequacy Level')

    for ax in axs:
        ax.set_xlabel('Adequacy Share')

    plt.tight_layout()

    plt.show();
    
    ### Nutrients Average Share vs Adequacy Share
    adequacy_shares = (hh_diet_adequacy >= 1).mean()
    adequacy_shares_75pct = (hh_diet_adequacy >= 0.75).mean()
    adequacy_shares_50pct = (hh_diet_adequacy >= 0.5).mean()

    # Calculate the average value of each column
    avg_shares = hh_diet_adequacy.mean()

    # Create a new DataFrame with the percentage over 1 and average values, indexed by column names
    adequacy_shares_summary = pd.DataFrame({'Average Share': avg_shares, 
                                            'Adequacy Level (Full)': adequacy_shares,
                                           'Adequacy Level (75%)': adequacy_shares_75pct,
                                           'Adequacy Level (50%)': adequacy_shares_50pct})
    adequacy_shares_summary = adequacy_shares_summary[adequacy_shares_summary.index.isin(nutrient_cols)]
    
    colors = ['green', 'orange', 'red']

    plt.figure(figsize=(10, 6))

    for i, (col, color) in enumerate(zip(adequacy_shares_summary.columns[1:], colors)):
        sns.regplot(x='Average Share', y=col, data = adequacy_shares_summary,
                    scatter= True, color = color, label = col)

    plt.xlabel('Average Share')
    plt.ylabel('Adequacy Share')
    plt.title('Nutrients Average Share vs Adequacy Share')
    plt.legend()
    plt.grid(True)
    plt.show();
    
    ### Displays the Nutrients with the Highest Average Share
    print('Nutrients with the Highest Average Share')
    display(adequacy_shares_summary.sort_values('Average Share', ascending = False).head())
    
    ### Adequacy Shares Across Regions
    hh_diet_adequacy.reset_index(inplace = True)
    hh_diet_adequacy['m'] = household_master['m']
    hh_diet_adequacy = hh_diet_adequacy.set_index(['i', 'm']).reset_index().set_index('i')
    
    fig, axs = plt.subplots(1, 3, figsize=(20, 5))
    adequacy_levels = [1, 0.75, 0.5]
    a_lvl_titles = ['100% Adequacy Level', '75% Adequacy Level', '50% Adequacy Level']
    colors = ['green', 'orange', 'red']

    plt.suptitle('Adequacy Shares Across Regions At Different Adequacy Levels')

    for i, a_lvl in enumerate(adequacy_levels):
        hh_diet_adequacy_by_region = hh_diet_adequacy.set_index('m').iloc[:, :len(nutrient_cols)]
        hh_diet_adequacy_by_region = (hh_diet_adequacy_by_region >= a_lvl).sum(axis = 1) / len(nutrient_cols)
        color = colors[i % len(colors)]
        axs[i].set_title(a_lvl_titles[i])
        sns.boxplot(x = hh_diet_adequacy_by_region.index,
                   y = hh_diet_adequacy_by_region.values, ax = axs[i])

    for ax in axs:
        ax.set_xlabel('Region')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    
    plt.show();

In [ ]:
def plot_nutrient_distributions(nutrient, hh_diet_adequacy):
    '''
    Description
    --------------------------------------------------
    This function plots the Household Adequacy Levels 
    distribution for a given nutrient

    Inputs
    --------------------------------------------------
    + nutrient : string; the nutrient that the
            distribution will be plotted for
    + hh_diet_adequacy : pandas dataframe; contains
            the nutrient shares where share equals
            (household nutrient consumption) / (hous-
            -ehold nutrition requirement)
    
    Outputs
    --------------------------------------------------
    + Plots the nutrient distribution
    '''
    
    data = hh_diet_adequacy[nutrient]

    # Creating a histogram with range excluding outliers
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    sns.histplot(data[(data >= lower_bound) & (data <= upper_bound)], stat = 'density', edgecolor='black')

    plt.xlabel('Adequacy Level')
    plt.title(f'Distribution of Household {nutrient} Adequacy Levels \n Level = (Household Nutrient Intake / Nutrient Requirement)')

    plt.show();  

## Interactivity Development

In [ ]:
def run_nutritional_adequacy_analysis(country, year, diet_refs = diet_refs):
    '''
    Description
    --------------------------------------------------
    This is a function that packages up all of the 
    functions in the notebook.

    Inputs
    --------------------------------------------------
    + country : string; country of interest
    + year : string; year of interest - in the form
            20XX-XX+1
    + diet_refs : pandas dataframe; dietary reference 
            intakes for different age-sex groups
    
    Outputs
    --------------------------------------------------
    + Interactive widget is displayed
    
    The following outputs used in 
    plot_nutrient_distributions_interactive():
    
    + hh_diet_adequacy : pandas dataframe; contains
            the nutrient shares where share equals
            (household nutrient consumption) / (hous-
            -ehold nutrition requirement)
    + nutrient_cols : numpy array; all of the
            column names of the nutrients of 
            interest
    '''
    country = country.lower()
    food_nutrients, food_prices, hh_chars, expenditures, hhc_sub, fp_sub, food_cols, fp_sub_avgs = get_data(country, year_range)
    
    hh_diet_adequacy, household_master, nutrient_cols = get_diet_adequacies(food_nutrients, food_prices, hh_chars, 
                        expenditures, hhc_sub, fp_sub, food_cols, fp_sub_avgs, diet_refs)
    
    get_plots(hh_diet_adequacy, household_master, nutrient_cols)
    return nutrient_cols, hh_diet_adequacy

In [ ]:
def plot_nutrient_distributions_interactive(nutrient_cols, hh_diet_adequacy):
    '''
    Description
    --------------------------------------------------
    This is a function that makes 
    plot_nutrient_distributions() interactive.

    Inputs
    --------------------------------------------------
    + hh_diet_adequacy : pandas dataframe; contains
            the nutrient shares where share equals
            (household nutrient consumption) / (hous-
            -ehold nutrition requirement)
    + nutrient_cols : numpy array; all of the
            column names of the nutrients of 
            interest
    
    Outputs
    --------------------------------------------------
    + Interactive widget is displayed
    '''
    widget = interactive(plot_nutrient_distributions,
                         nutrient = widgets.Dropdown(options = nutrient_cols, 
                                                     description = "Nutrient"),
                         hh_diet_adequacy = widgets.fixed(hh_diet_adequacy))
    display(widget)